# TextAttack End-to-End

This tutorial provides a broad end-to-end overview of training, evaluating, and attacking a model using TextAttack.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QData/TextAttack/blob/master/docs/2notebook/0_End_to_End.ipynb)

[![View Source on GitHub](https://img.shields.io/badge/github-view%20source-black.svg)](https://github.com/QData/TextAttack/blob/master/docs/2notebook/0_End_to_End.ipynb)

## Training

First, we're going to train a model. TextAttack integrates directly with [transformers](https://github.com/huggingface/transformers/) and [datasets](https://github.com/huggingface/datasets) to train any of the `transformers` pre-trained models on datasets from `datasets`. 

Let's use the SNLI textual entailment dataset: it's relatively short (in word count, at least), and showcases a lot of the features of `textattack train`. Let's take a look at the dataset using `textattack peek-dataset`:

In [1]:
!textattack peek-dataset --dataset-from-huggingface snli

Reusing dataset snli (/p/qdata/jy2ma/.cache/textattack/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)
textattack: Loading datasets dataset snli, split train.
textattack: Number of samples: 550152
textattack: Number of words per input:
textattack: 	total:   11150480
textattack: 	mean:    20.27
textattack: 	std:     6.95
textattack: 	min:     4
textattack: 	max:     112
textattack: Dataset lowercased: False
textattack: First sample:
Premise: A person on a horse jumps over a broken down airplane.
Hypothesis: A person is training his horse for a competition. 

textattack: Last sample:
Premise: A man is surfing in a bodysuit in beautiful blue water.
Hypothesis: On the beautiful blue water there is a man in a bodysuit surfing. 

textattack: Found 4 distinct outputs.
textattack: Most common outputs:
	 0      (183416)
	 2      (183187)
	 1      (182764)
	 -1     (785)


The dataset looks good! It's not lowercased already, so we'll make sure our model is cased. Looks like there are some missing (-1) labels, so we need to filter those out. The longest input is 114 words, so we can cap our maximum sequence length (`--model-max-length`) at 128.

We'll train [`distilbert-base-cased`](https://huggingface.co/transformers/model_doc/distilbert.html), since it's a relatively small model, and a good example of how we integrate with `transformers`.

So we have our command:

```bash
textattack train                      \ # Train a model with TextAttack
    --model distilbert-base-cased     \ # Using distilbert, cased version, from `transformers`
    --dataset snli                    \ # On the SNLI dataset
    --model-num-labels 3              \ # That has 3 labels
    --filter-train-by-labels 0 1 2    \ # And filter -1 label from train and test
    --filter-eval-by-labels 0 1 2
    --model-max-length 128            \ # With a maximum sequence length of 128
    --per-device-train-batch-size 128 \ # And batch size of 128
    --num-epochs 2                    \ # For 2 epochs (since SNLI is large)
```

Now let's run it:

In [2]:
!textattack train --model-name-or-path distilbert-base-cased --dataset snli --model-num-labels 3 --filter-train-by-labels 0 1 2 --filter-eval-by-labels 0 1 2 --model-max-length 128 --per-device-train-batch-size 128 --num-epochs 2

textattack: Loading transformers AutoModelForSequenceClassification: distilbert-base-cased
Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-ca

## Evaluation

We successfully fine-tuned `distilbert-base-cased` for 3 epochs. Now let's evaluate it using `textattack eval`. This is as simple as providing the path to the pretrained model to `--model`, along with the number of evaluation samples. `textattack eval` will automatically load the evaluation data from training:

In [15]:
!textattack eval --num-examples 10000 --model ./outputs/2021-06-01-09-32-01-176023/best_model/ --dataset-from-huggingface snli --dataset-split test

Reusing dataset snli (/p/qdata/jy2ma/.cache/textattack/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)
textattack: Loading datasets dataset snli, split test.
textattack: Got 10000 predictions.
textattack: Correct 8684/10000 (86.84%)


Awesome -- we were able to train a model up to 86.84% accuracy on the test dataset – with only a single command!

## Attack

Finally, let's attack our pre-trained model. We can do this the same way as before (by providing the path to the pretrained model to `--model`). For our attack, let's use the "TextFooler" attack recipe, from the paper ["Is BERT Really Robust? A Strong Baseline for Natural Language Attack on Text Classification and Entailment" (Jin et al, 2019)](https://arxiv.org/abs/1907.11932). We can do this by passing `--recipe textfooler` to `textattack attack`.

> *Warning*: We're printing out 1000 examples and, if the attack succeeds, their perturbations. The output of this command is going to be quite long!


In [22]:
!textattack attack --recipe textfooler --num-examples 1000 --model ./outputs/2021-06-01-09-32-01-176023/best_model/ --dataset-from-huggingface snli --dataset-split test

Reusing dataset snli (/p/qdata/jy2ma/.cache/textattack/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)
textattack: Loading datasets dataset snli, split test.
textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  univ

Looks like our model was 87.0% successful (makes sense - same evaluation set as `textattack eval`!), meaning that TextAttack attacked the model with 870 examples (since the attack won't run if an example is originally mispredicted). The attack success rate was 97.13%, meaning that TextFooler failed to find an adversarial example only 2.87% of the time.


## Conclusion

That's all, folks! We've learned how to train, evaluate, and attack a model with TextAttack, using only three commands! 😀